# Seasonal Regression

The electricity demand series shows daily, weekly, and annual oscillations.  At a short time scale, I aim to capture the first two of these.

The approaches I've seen suggest Fourier Series, linear regression, and Seasonal ARIMA.
(I got quite stuck on how to detrend the series in a global fashion.)
I will focus on building models on the last two weeks of data, with the goal of predicting electricity demand based on temperature, time of day, and day of week.

## Hyndman's Multiple Seasonal Exponential Smoothing

This follows Rob Hyndman's approach towards multi-seasonal exponential smoothing.  (This generalizes the apparently well-known Holt-Winters smoothing).
His analysis includes forecasts of electricity generation, based on utility data (from well over 10 years ago).

I chose to follow this model since initial attempts at ARIMA rely on removing the seasonality, and I had hoped to just follow best practice with existing libraries.  Initial naive methods gave complete crap, and failed to remove the seasonal pattern, or even worse imposed one.  An initial attempt at Fourier filtering on over a year of data also left a 

Hyndman also seems to be a known author within the field of econometric time-series forecasting.  

The original model for a variable $y_t$, with seasonal pattern with period $m$ is
\begin{align}
  y_t &= l_{t-1}+b_{t-1} +s_{t-m} +\epsilon_t\\
  l_t &= l_{t-1} + \alpha\epsilon_t\\
  b_t &= b_{t-1} + \beta\epsilon_t\\
  s_{t} = s_{t-m} + \gamma \epsilon_t
\end{align}
where $l_t$, b_t,s_t$ are the level, trend and seasonal patterns respectively.
The noise is Gaussian and obeys
$E[\epsilon_t]=0, E[\epsilon_t\epsilon_s]=\delta_{ts}\sigma^2$, and $\alpha,\beta,\gamma$ are constants between zero and one.  (He notes that $m+2$ estimates must be made for the initial values of the level, trend and seasonal pattern).

Hyndman's model allows multiple seasons, and allows the sub-seasonal terms to be updated more quickly than once per large season.  In utility data, the short season is the daily oscillation, while the longer season comes from the weekly oscillation induced by the work week.  For hourly data, the daily cycle has length $m_1=24$, with the weekly cycle taking $m_2=168$.  The ratio between them is $k=m_2/m_1=7.$  The number of seasonal patterns is $r\le k$.  

(I'm going to change Hyndman's notation to use $\mathbf{I}$ to denote indicator/step functions).
\begin{align}
  y_t &= l_{t-1}+b_{t-1} +\sum_{i=1}^r \mathbf{I}_{t,i}s_{i,t-m_1} +\epsilon_t\\
  s_{i,t} = s_{i,t-m_1} + \sum_{j=1}^r\left(\gamma_{ij}\mathbf{I}_{t,j}\right) \epsilon_t  (i=1,2,\ldots,r)
  l_t &= l_{t-1} + b_{t-1}+\alpha\epsilon_t\\
  b_t &= b_{t-1} + \beta\epsilon_t\\
\end{align}
Here the indicator functions $\mathbf{I}_{t,i}$ are unity if $t$ is in the seasonal pattern $i$, and zero otherwise.  For utility data, this will probably be weekday and holiday/weekend.  Here $\gamma_{ij}$ denotes how much one seasonal pattern is updated based on another---Hyndman proposes a number of restrictions on these parameters.

I will extend this to include an external variables for the deviation above a given temperature, so that $y_t\rightarrow y_t+\tau_p\Theta(T_t-T_p)+\tau_{n}\Theta(T_n-T_t)$.

He suggests using the first four weeks of data to estimate the parameters, by minimizing the squared error of the one-step ahead forecast.  Apparently maximum likelihood estimation was not recommended (10 years ago).

So how to fit the parameters?  A really simple approach would be gradient descent?  Intuitively, the level is the average value, the bias is the average gradient.  The seasonality is the average seasonal pattern.  (This is the dumb STL decomposition used earlier?)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from get_weather_data import convert_isd_to_df, convert_state_isd
from EBA_util import remove_na, avg_extremes

from numpy import pi,e

%matplotlib inline

In [2]:
try:
    df_joint=pd.read_csv('data/pdx_joint.txt',
        index_col=0, parse_dates=True)
    print('Read in PDX Frame from file')
    dem=df_joint['Demand'].copy()
    temp=df_joint['Temp'].copy()
    fore=df_joint['Forecast'].copy()
except:
    air_df = pd.read_csv('data/air_code_df.gz')
    #Just get the weather station data for cities in Oregon.
    df_weather=convert_state_isd(air_df,'OR')
    #Select temperature for Portland, OR
    msk1=np.array(df_weather['city']=='Portland')
    msk2=np.array(df_weather['state']=='OR')
    df_pdx_weath=df_weather.loc[msk1&msk2]
    #get electricity data for Portland General Electric
    df_eba=pd.read_csv('data/EBA_time.gz',index_col=0,parse_dates=True)
    msk=df_eba.columns.str.contains('Portland')
    df_pdx=df_eba.loc[:,msk]
    msk1=  df_pdx.columns.str.contains('[Dd]emand') 
    dem=df_pdx.loc[:,msk1]
    #Make a combined Portland Dataframe for demand vs weather.
    df_joint=pd.DataFrame(dem)
    df_joint=df_joint.join(df_pdx_weath)
    temp=df_joint['Temp']
    df_joint['TempShift']=150+abs(temp-150)
    df_joint = df_joint.rename(columns={df_joint.columns[0]:'Demand',
             df_joint.columns[1]:'Forecast'})
    df_joint.to_csv('data/pdx_joint.txt')

    # msk=  df_eba.columns.str.contains('Demand') \
    #     & df_eba.columns.str.contains('demand')
    # dem=df_pdx.loc[:,msk]
    # #Make a combined Portland Dataframe for demand vs weather.
    # df_joint=pd.DataFrame(dem)
    # df_joint=df_joint.join(df_pdx_weath)
    # temp=df_joint['Temp']
    # df_joint['TempShift']=150+abs(temp-150)
    # df_joint=df_joint.rename(columns={df_joint.columns[0]:'Demand',df_joint.columns[1]:'Forecast'})
    # df_joint.to_csv('data/pdx_joint.txt')


Read in PDX Frame from file


In [3]:
#clean up data, remove NA
dem = remove_na(dem)
dem = avg_extremes(dem)
temp = avg_extremes(remove_na(temp))

Number of extreme values 0. Number of zero values 148


Number of NA values 56


Number of extreme values 1. Number of zero values 3


Number of NA values 156


In [321]:
'a' in ['a','b']


True

In [164]:
#from EBA_seasonal import multiseasonal as ms
#Class for Multiseasonal model.
class ms:
    def __init__(self, l=0,b=0,s=np.zeros((2,24)),  alpha=0.1,beta=0.1,gamma=0.1*np.ones((2,2))):
        self.l=l
        self.b=b
        self.s=s
        self.alpha=alpha
        self.beta=beta
        #easiest to initialize via a single matrix.
        self.g00 = gamma[0,0]
        self.g01 = gamma[0,1]
        self.g10 = gamma[1,0]
        self.g11 = gamma[1,1]

    def gamma(self):
        gamma=np.array([[self.g00,self.g01],[self.g10,self.g11]])
        return gamma


    def fit_init_params(self,y,ninit=4*24*7):
         """fit_init_params(y)
         Fits initial parameters for Hyndman's multi-seasonal model to
         hourly electricity data.
         (My guess on how to do this, similar to naive STL method used in 
         statstools.timeseries)
         Finds level, bias and seasonal patterns based on first 4 weeks of data.  
         """
         ysub = y[0:ninit]
         yval = ysub.values
         ##average value
         self.l = np.mean(yval)
         ##average shift
         self.b = (yval[ninit-1]-yval[0])/ninit
         ##remove mean pattern, subtract off level, and linear trend.
         ysub = ysub-self.l-self.b*np.arange(ninit)
         #mean seasonal pattern.
         #second seasonal pattern is for weekends, with days
         #Saturday/Sunday have dayofweek equal to 5 and 6.
         #make a mask to select out weekends.
         s2 = ysub.index.dayofweek >=5
         #select out weekends, and regular days. 
         y_end = ysub[s2]
         y_week= ysub[~s2]
         n1 = int(len(y_week)/24)
         n2 = int(len(y_end)/24)
         self.s = np.zeros((2,24))
         for n in range(n1):
              self.s[0,:] = self.s[0,:]+y_week[n*24:(n+1)*24]/n1
         for n in range(n2):
              self.s[1,:] = self.s[1,:]+y_end[n*24:(n+1)*24]/n2

    def predict_correct_onestep(self,yhat,ypred,t):
         """predict_correct_onestep
         Updates time parameters based on differences between predicted 
         and measured.  Also predicts next step based on current values. 
         (Not fixing the model parameters for update strength!)
         Work in Progress
         """
         eps=(yhat-ypred)
         #find seasonal patterns.  
         m1 = t.hour                     
         mr  = int(t.dayofweek>=5)
         nmr = int(t.dayofweek<5)         
         #Original version with bias in there too?
         self.l = self.l + self.b + self.alpha*eps
         self.b = self.b + self.beta*eps
        
         #ynew = self.l + self.s[mr,m1]         
         # self.l = self.l +self.alpha*eps         
         # self.b = self.b + self.beta*eps
         # ynew = self.l  + self.s[mr,m1]
         
         #update row, and hour
         ds = np.dot(self.gamma(),np.array([nmr,mr]))*eps
         self.s[:,m1] = self.s[:,m1]+ds

         #predict the next value given the updated parameters
         ynew = self.l + self.s[mr,m1]
         return ynew

    def STL_onestep(self,y,ninit=4*24*7):
         """STL_onestep
         Generates initial parameters, and then predicts remainder
         of series for input data y.  
         """
         self.fit_init_params(y,ninit=ninit)
         ypred = np.zeros(len(y))         
         t0=y.index[0]
         m1 = t0.dayofweek>=5
         m2 = t0.hour
         
         ti = y[:ninit].index
         msk=ti.dayofweek>=5
         ypred[:ninit] = self.l+self.b*np.arange(ninit) \
                  + self.s[msk.astype(int),ti.hour.values]

         for i in range(ninit,len(y)):
             ynew=self.predict_correct_onestep(y[i],
                     ypred[i-1],y.index[i])
             ypred[i] = ynew
         # if i%(24*7) ==0:
         #         print("l: {} b: {}\n".format(str(l),str(b)))
         #         print(s,"\n")
         ypred=pd.Series(ypred,index=y.index)         
         return ypred

    def predict_correct_dayahead(self,y):
         """predict_correct_dayahead
         Predict day-ahead demand given previous parameters.
         Then update parameters given true demand.
         """
         t0=y.index
         m1 = t0.dayofweek>=5
         m1_n = t0.dayofweek<5         
         m2 = t0.hour
         trend=self.l+self.b*np.arange(len(y))
         season=self.s[m1.astype(int),m2]
         ypred = trend+season

         eps = y-ypred
         eps_l = np.mean(eps)
         self.l = self.l + self.alpha*eps_l
         eps=eps-eps_l
         eps_b = (eps[-1]-eps[0])/len(eps)
         self.b = self.b + self.beta*eps_b
         eps=eps-eps_b*np.arange(len(eps))
         ds = np.dot(self.gamma(),np.array([m1_n,m1]))*[eps,eps]
         self.s = self.s + ds
         ytot=pd.Series(ypred,index=y.index)
         return ytot

    def STL_dayahead(self,y,ninit=4*24*7):
         """STL_dayahead
         Predict day-ahead demand given previous parameters.
         Then update parameters given true demand.
         """
         self.fit_init_params(y,ninit=ninit)
         t0=y.index[0]
         m1 = t0.dayofweek>=5
         m2 = t0.hour
         ypred = np.zeros(len(y))
         ti = y[:ninit].index
         msk=ti.dayofweek>=5
         ypred[:ninit] = self.l+self.b*np.arange(ninit) \
                  + self.s[msk.astype(int),ti.hour.values]
                  
         for i in range(int(ninit/24),int(len(y)/24)):
             tslice = slice(i*24,(i+1)*24)
             ypred[tslice] = self.predict_correct_dayahead(y[tslice])
             
         # if i%(24*7) ==0:
         #         print("l: {} b: {}\n".format(str(l),str(b)))
         #         print(s,"\n")
         ypred=pd.Series(ypred,index=y.index)         
         return ypred

    def optimize_param(self,y,ninit=4*24*7,rtol=0.01,\
                        eta=0.01,lr=0.05,nmax=100):
        """optimize_param
        Use gradient descent to find optimum parameters for learning 
        rates alpha,beta,gamma.  Wait till all of their values are 
        settled to a relative tolerance.
        Cost is root Mean Square Error over whole time series.
        Currently tries to predict day ahead.  
        """
        self.fit_init_params(y)
        #Super clunky way of specifiying names.
        #Why did I think this was superior?
        names=['alpha','beta','g00','g01','g10','g11']
        pred0 = self.STL_dayahead(y,ninit=ninit)
        J    = self.rmse(y[ninit:],pred0[ninit:])
        Ni=0
        #loop over iterations
        for i in range(nmax):
            dJ_max=0
            #for each name, tweak the model's variables.
            eta=eta*0.99
            for n in names:
                p0=self.__getattribute__(n)            
                self.__setattr__(n,p0*(1+eta))
                #print(n,p0,p0*(1+eta))
                pred=self.STL_dayahead(y,ninit=ninit)
                J2=self.rmse(y[ninit:],pred[ninit:])
                dJ = np.abs((J2-J)/J)
                # if (debug):
                #     print('J,J2,p',J,J2,p0)
                #actually update 
                #p = p0-lr*(J2-J)/(eta*p0)
                p = p0+lr*dJ/(eta*p0)
                if (p>1):
                    print('param {} >1(!): {}'.format(n,p))
                    print('J,J2,p',J,J2,p0)
                    p=min(p0+0.5*(1-p0),0.99)
                elif(p<0): 
                    print('param {} <0(!): {}'.format(n,p))
                    print('J,J2,p',J,J2,p0)
                    p=0.5*p0
                self.__setattr__(n,p)
                J=J2
                dJ_max=max(dJ,dJ_max)
            Ni+=1       
            if (dJ_max<rtol):
               print("Hit tolerance {} at iter {}".format(dJ,Ni))
               plot_pred([pred,y],['Predicted','Actual'])               
               return pred
            if(Ni%10==0):
                print("Cost, Old Cost = {},{}".format(J,J2))
                plot_pred([pred,y],['Predicted','Actual'])
                for n in names:
                    p0=self.__getattribute__(n)
                    print(n,p0)

        print("Failed to hit tolerance after {} iter\n".format(iter))
        print("Cost:",J,J2)
        return pred 

    def root_optimize_param(self,y,ninit=4*24*7,rtol=0.01,\
                        eta=0.2,lr=0.1,nmax=100):
        """root_optimize_param
        Use simple root-finding to find the minimum error to optimize  
        the parameters for alpha,beta,gamma.
        
        Based on data, and for chosen func, which predicts all values 
        of y.  Pick one with minimum Mean Squared Error.
        """
        self.fit_init_params(y)
        #Specify parameters to search over.  
        names=['alpha','beta','g00','g01','g10','g11']
        pred0 = self.STL_dayahead(y,ninit=ninit)
        J    = self.rmse(y[ninit:],pred0[ninit:])
        Ni=0
        #loop over iterations
        for i in range(nmax):
            dJ_max=0
            #for each name, tweak the model's variables.
            eta=eta*0.99
            for n in names:
                p0=self.__getattribute__(n)            
                self.__setattr__(n,p0*(1+eta))
                print(n,p0,p0*(1+eta))
                pred=self.STL_dayahead(y,ninit=ninit)
                J2=self.rmse(y[ninit:],pred[ninit:])
                dJ = np.abs((J2-J)/J)
                J=J2
                dJ_max=max(dJ,dJ_max)
                #actually update 
                self.__setattr__(n,p)
            Ni+=1       
            if (dJ_max<rtol):
               print("Hit tolerance {} at iter {}".format(dJ,Ni))
               plot_pred([pred,y],['Predicted','Actual'])               
               return pred
            if(Ni%10==0):
                print("Cost, Old Cost,Max = {},{},{}".format(J,J2,dJ_max))
                plot_pred([pred,y],['Predicted','Actual'])   
        print("Failed to hit tolerance after {} iter\n".format(iter))
        print("Cost:",J,J2)
        return pred 


    def rmse(self,x,y):
        """compute mean_square_error"""
        z = np.sum( (x-y)*(x-y))/len(x)
        z = np.sqrt(z)
        return z
#End of Class
def scale_data(D,T):
    """scale_data
    Scales both demand and temperature to have zero mean, 
    unit standard deviation.
    Returns scaled data, as well as means, and standard deviations.
    """
    scaling_df=pd.DataFrame(columns=['dem','temp'],index=['mu','std'])
    scaling_df.loc['mu','dem']=D.mean()
    scaling_df.loc['std','dem']=D.std()
    scaling_df.loc['mu','temp']=T.mean()
    scaling_df.loc['std','temp']=T.std()
    T_scale=(T-T.mean())/T.std()
    D_scale=(D-D.mean())/D.std()
    return D_scale,T_scale,scaling_df

def plot_pred(series_list,label_list):
    """make plot to compare fitted parameters"""
    for s,l in zip(series_list,label_list):
        plt.plot(s,label=l)    
    plt.legend()
    plt.show()


The following code tries to optimize the hyperparameters.  It seems to lock onto essentially a persistence model.  The best guess is to just use yesterday's electricity usage to predict today's. (Considering that was going to be my benchmark for simple and effective, this is somewhat disheartening.

I tried this numerically rather than via directly coding up gradient descent, since the closed form derivatives become nasty polynomials in the parameters $(\alpha,\beta,\gamma)$. The previous values $y_{t-1}$ also depend on the choice of parameters, so when you consider the derivative of the cost function
$\partial_\alpha J = T^{-1}\sum_t \partial_\alpha(y_t-\hat{y}_t)$ there is a nested sum of terms (since the parameter estimates rely on the previous errors too).

In [165]:
start = 24*7*30
end =  start+24*7*4+24*7*20
dem_sub = dem[start:end]
temp_sub = temp[start:end]
fore_sub = fore[start:end]

alpha=0.3
beta=0.2
gamma=0.04*np.array([[1,0.4],[0.4,1]])
global icount
icount=0
ms_model=ms(alpha=alpha,beta=beta,gamma=gamma)
ytot=ms_model.STL_onestep(dem_sub)

alpha2=0.1
beta2=0.1
gamma2=0.1*np.array([[1,0.2],[0.2,1]])
ms_model2=ms(alpha=alpha2,beta=beta2,gamma=gamma2)
ytot2=ms_model2.STL_dayahead(dem_sub)

In [166]:
debug=True
ms_model2=ms(alpha=alpha2,beta=beta2,gamma=gamma2)
ypred=ms_model.optimize_param(dem_sub,rtol=1E-6,nmax=10)

alpha 0.99
beta 0.985147928986
g00 0.99
g01 0.99
g10 0.991569674802
g11 0.99
Failed to hit tolerance after <built-in function iter> iter

Cost: 168.316319889 168.316319889


param g11 >1(!): 1.0353776240021706
J,J2,p 169.695206208 168.316319889 0.99
Cost, Old Cost = 168.31631988930127,168.31631988930127


param g01 >1(!): 1.0390300512948585
J,J2,p 168.228636805 169.705633112 0.99


param g00 >1(!): 1.0032790565819971
J,J2,p 168.416992255 168.228636805 0.997077777089


param alpha >1(!): 1.0001353097919528
J,J2,p 168.298348873 168.603794575 0.99


...

param alpha >1(!): 16.60663284451791
J,J2,p 1915225.20041 196.780109595 0.313068146291


param g11 >1(!): 1245610.8252170554
J,J2,p 194.119107324 1915225.20041 0.04


param g10 >1(!): 10.286301217387415
J,J2,p 188.002208391 194.119107324 0.016


param g01 >1(!): 26.414105547722347
J,J2,p 173.493118009 188.002208391 0.016


param g00 >1(!): 2.199168306879345
J,J2,p 176.511574126 173.493118009 0.04


672

In [350]:
names=['alpha','g00']

val=[]
for n in names:
    val.append(ms_model.__getattribute__(n))

blah = [ms_model.__getattribute__(name) for name in names]
?ms_model.__setattr__

In [343]:
dir(ms_model)

['STL_dayahead',
 'STL_onestep',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'alpha',
 'b',
 'beta',
 'fit_init_params',
 'g00',
 'g01',
 'g10',
 'g11',
 'gamma',
 'l',
 'mse',
 'optimize_param',
 'predict_correct_dayahead',
 'predict_correct_onestep',
 's']

In [248]:
per=slice(None,'2016-05-15')
plt.plot(ytot[per],'b',label='My Predicted')
plt.plot(fore_sub[per],'g',label='Actual Predicted')
plt.plot(dem_sub[per],'r',label='Actual')
plt.legend()
plt.show()

In [306]:
plt.plot(np.cumsum(ytot-fore))
plt.show()

In [454]:
l,b,s=ms.fit_init_params(dem_sub,ninit=4*24*7)
plt.plot(s.T)
plt.show()

20 8


# Thresholded Temperature Model

Let's just try to build a linear model for the temperature on top of this.
I'll assume that heating/cooling might have different coefficients, so the temperature component of the model at time $t$ is
\begin{equation}
D_t =  a_0+ a_+[T_t-T_{+}]_{+} + a_-[T_{-}-T_t]_{+},
\end{equation}
where $[f]_+=f$ if $f>0$, and is zero otherwise.  If we optimize the mean square
error, then the components can be found by solving for the values that minimize the derivatives.
If criteria is mean square error, then can solve directly for parameters
$J = sum_t[\hat{D}_t-D_t)^2$, where $\hat{D}_t$ is the true value.
We must solve $\partial J/\partial \alpha = 0 \rightarrow \sum_t \frac{\partial D_t}{\partial\alpha}(\hat{D}_t-D_t) $

Those conditions are
\begin{align}
    \sum_t(\hat{D}_t-D_t)=0  \qquad (a_0)\\
    \sum_{t\in T_\pm}[T_t-T_\pm]_\pm(\hat{D}_t-D_t)=0  \qquad (a_\pm)\\
    \sum_{t\in T_\pm}(\pm a_{\pm})(\hat{D}_t-D_t)=0  \qquad (T_\pm)\\
\end{align}


(First OOP class - absolutely ridiculous)  Will just use a pandas Dataframe - want a named set of numbers, with defined operations.  Already here.  

In [34]:
#Experimenting with OOP for making "vector" of parameters with named labels.
#Feels daft - if there's a less stupid way, I'll try to fix this.  (Just use a Dataframe!?)
#Initial attempts at getting smarter initialization (with arguments to make a dict just returned empty.
# class param_vec(dict):
#     """Class for model parameters.
#     Stores parameters in dict, with arithmatic operations.
#     """
#     #Define elementwise subtraction/addition
#     def __add__(self,x):
#         y = self.copy()
#         if isinstance(x,(int,float)):
#             for i,v in self.items():
#                 y[i]=self[i]+x
#         else:
#             for i,v in self.items():
#                 y[i]=self[i]+x[i]
#         return param_vec(y)

#     #Define elementwise subtraction/addition
#     def __radd__(self,x):
#         return param_vec.__add__(self,x)
    
#     #Define elementwise subtraction.
#     def __sub__(self,x):
#         y=self.copy()
#         if isinstance(x,(int,float)):
#             for i,v in self.items():
#                 y[i]=v-x
#         else:        
#             for i,v in self.items():
#                 y[i]=v-x[i]
#         return param_vec(y)
                
#     #Define elementwise subtraction/addition
#     def __rsub__(self,x):
#         return param_vec.__sub__(self,x)
                
#     #Define elementwise subtraction.
#     def __truediv__(self,x):
#         y=self.copy()
#         if isinstance(x,(int,float)):
#             for i,v in self.items():
#                 y[i]=v/x
#         else:        
#             for i,v in self.items():
#                 y[i]=v/x[i]
#         return param_vec(y)
                
#     #Define elementwise subtraction.
#     def __mul__(self,x):
#         y=self.copy()
#         if isinstance(x,(int,float)):
#             for i,v in self.items():
#                 y[i]=v*x
#         else:        
#             for i,v in self.items():
#                 y[i]=v*x[i]
#         return param_vec(y)
                
#     #Define elementwise subtraction.
#     def __rmul__(self,x):
#         return param_vec.__mul__(self,x)

def param_vec(names,vals):
    p=pd.Series(vals,index=names)
    return p

#might extend to contain all of the modelling stuff?          

In [94]:
dem_sub  = dem[0:24*7*1]
temp_sub = temp[0:24*7*1]

D=dem_sub
T=temp_sub
Dr = np.max(D)-np.min(D)
Tr = np.max(T)-np.min(T)

pnames=['a0','ap','an','Tp','Tn']
pval=[np.mean(D),0.5*Dr/Tr,0.5*Dr/Tr,200,100]

Tmodel=just_temp_model(names=pnames,vals=pval)



In [133]:
class just_temp_model:
    def __init__(self,names=[],vals=[]):
        self.param= pd.Series(vals,index=names)

    def temp_model(self,T):
        """temp_model(self,T)
        Tries to fit linear model for electricity demand to temperature.
        Initially tried to allow thresholding, and different slopes for heating/cooling.  
        Will now just try simpler linear model a_p|T-T_p|.
        """ 
        m1 = T>self.param['Tp']
        m2 = T<self.param['Tn']
        y=np.zeros(T.shape)
        y[m1] = (T[m1]-self.param['Tp'])*self.param['ap']
        y[m2] = (self.param['Tn']-T[m2])*self.param['an']
        y=y+self.param['a0']
        #y = param['a0']+ param['ap']*np.abs(T-param['Tp'])
        y=pd.Series(y,name='Predicted Demand',index=T.index)
        return y

    def temp_model_grad(self,D,Dhat,T):
        """temp_model_grad
        Compute gradients of model w.r.t. parameters.
        Assumes loss-function is mean-square.
        Dhat - measured demand
        D    - predicted demand
        T    - measured temperature
        """
        m1 = T>self.param['Tp']
        m2 = T<self.param['Tn']
        Nt = len(T)
        Derr=D-Dhat
        #initialize with zeros
        dparam=param_vec(self.param.keys(),np.zeros(len(self.param)))
        dparam['a0'] = np.sum(Derr)/Nt
        #Single model
        # dparam['ap'] = np.sum( np.abs(T-param['Tp'])*Derr)/Nt
        # dparam['Tp'] = -np.sum(np.sign(T-param['Tp'])*param['ap']*Derr)/Nt
        #Double thresholded model
        dparam['ap'] = np.sum( np.abs(T[m1]-self.param['Tp'])  \
                     *(Derr[m1]))/Nt
        dparam['an'] = np.sum( (self.param['Tn']-T[m2])*(Derr[m2]))/Nt
        dparam['Tp'] = -self.param['ap']*np.sum(Derr[m1])/Nt
        dparam['Tn'] =  self.param['an']*np.sum(Derr[m2])/Nt    
        return dparam
    
    def param_fit(self,Dhat,T,alpha=0.1,rtol=1E-4,nmax=200):
        """Try to fit linear threshold model of demand to temperature.
            D - demand data
            T - temperature data
            Fits model of form:
            D ~ a_0+ a_p[T-T_p]_+ + a_n[T_n-T]_+,
            where [f]_+ =f for f>0, and 0 otherwise.

            Just use simple gradient descent to fit the model.
        """
        #make parameter estimates
        Dr = np.max(Dhat)-np.min(Dhat)
        Tr = np.max(T)-np.min(T)
        param_names=['a0','ap','an','Tp','Tn']
        param_vals=[np.mean(Dhat), 0.5*Dr/Tr, 0.5*Dr/Tr,
        np.mean(T), np.mean(T)]
        self.param=param_vec(param_names,param_vals)
        Dpred = self.temp_model(T)
        J=np.sum((Dpred-Dhat)**2)/len(Dhat)
        print('Init cost',J)
        plot_pred(Dpred,Dhat,T)
        print('Param:',self.param,"\n")    

        Ni=0
        for i in range(nmax):
            dparam=self.temp_model_grad(Dpred,Dhat,T)
            self.param=self.param-alpha*dparam
            Dpred=self.temp_model(T)
            J2=J        
            J=np.sum((Dpred-Dhat)**2)/len(Dhat)
            err_change=abs(1-J2/J)
            Ni+=1
            if (err_change<rtol):
               print("Hit tolerance {} at iter {}".format(
               err_change,Ni))
               plot_pred(Dpred,Dhat,T)                      
               return Dpred
            if(Ni%100==0):
                print("Cost, Old Cost = {},{}".format(J,J2))
                print('Param:',self.param)
                print('Param_grad:',dparam)
                print("Mean param Change {} at iter {}".format(err_change,Ni))
                plot_pred(Dpred,Dhat,T)
        print("Failed to hit tolerance after {} iter\n".format(iter))
        print("Cost:",J,J2)
        return Dpred 

    def grad_check(self,Dhat,T):
        """grad_check(Dhat,T)
        Check numerical gradients against finite difference.
            D - demand data
            T - temperature data
        """
        #make parameter estimates
        Dr = np.max(Dhat)-np.min(Dhat)
        Tr = np.max(T)-np.min(T)
        param_names=['a0','ap','an','Tp','Tn']
        #param_vals=300*np.random.random(size=5)
        param_vals=[np.mean(D),0.5*Dr/Tr,0.5*Dr/Tr,200,250]        
        self.param=param_vec(param_names,param_vals)
        print(self.param)
        Dpred = self.temp_model(T)    
        dparam=self.temp_model_grad(Dpred,Dhat,T)    
        J=0.5*np.sum((Dpred-Dhat)**2)/len(Dhat)
        eps=.1
        param2 = self.param.copy()
        for name,val in param2.items():
            self.param = param2.copy()
            self.param[name]=val+eps
            Dpred = self.temp_model(T)
            J2=0.5*np.sum((Dpred-Dhat)**2)/len(Dhat)
            numgrad = (J2-J)/eps
            print(name,numgrad,dparam[name],self.param[name])

#End of temp_model class.
#Needed to scale the data to get decent answers.  



In [182]:
dem_sub  = dem[0:24*7*4]
temp_sub = temp[0:24*7*4]
#D,T,scale_df=scale_data(dem_sub,temp_sub)
Tmodel=just_temp_model(names=pnames,vals=pval)
Tmodel.grad_check(dem_sub,temp_sub)

a0    1.831868e-16
ap    4.540773e+00
an    4.540773e+00
Tp    2.000000e+02
Tn    2.500000e+02
dtype: float64
a0 -2233.0354153411463 -2233.08541539 0.1
ap -76396.42076404765 -90916.6982743 4.64077253219
an -74030.27359302621 -74171.064218 4.64077253219
Tp 3705.3657396975905 6423.67484413 200.1
Tn 130837.66448286362 -6434.87400116 250.1


In [177]:
np.sum(temp_sub<Tmodel.param['Tn'])

308

In [ ]:
# Fourier Series

Another approach that I've seen to seasonality is to just use a Fourier series.
This is similar to an approach I was using based on trying to filter the Fourier tranformed data.
This method tries to fit annual, weekly and daily oscillations to the data. 


In [56]:
#Now to do some simple Fourier Series fitting too.
class fourier_model:
    def total_fourier_series(self,D,n_max=[2,4,4]):
        """fourier_series
        Fits pandas time series D, with Fourier series. 
        Uses Pandas DateTimeIndex for times.
        Produces fourier series with annual, daily and weekly oscillations to fourier series.
        Computes coefficients, and then series.  Returns both
        D - demand (values to be fitted)
        T - DatetimeIndex
        n_max - maximum number of coefficients

        Note:Misses Holidays.
        """
        T=D.index
        T_dayofyear = T.dayofyear.values
        T_dayofweek = T.dayofweek.values
        T_hour = T.hour.values
        Tfit = [T_dayofyear/365,
        (T_dayofweek*24+T_hour)/168,
        T_hour/24]
        periods=[365,168,24]
        Nt = len(D)
        ftot = np.zeros(Nt)
        coeff=[[np.sum(D)/Nt,0]]    
        for i in range(3):
            if n_max[i]>0:
                 ci    = self.fit_fourier_series(D,Tfit[i],n_max[i])
                 ftot += self.fourier_series(ci,Tfit[i])             
            else:
                ci=None
            coeff.append(ci)
        #add on constant    
        ftot+= coeff[0][0]
        ftot=pd.Series(ftot,index=T)         
        return ftot,coeff

    def fit_fourier_series(self,D,T,nmax):
        """Fits the Fourier series to data D, on times T,
        and returns parameters.
        """
        Nt = len(D)
        #initial zero coefficients
        coeff=[]
        for n in range(1,nmax+1):
            an= 2*np.sum(np.cos(2*pi*n*T)*D)/Nt
            bn= 2*np.sum(np.sin(2*pi*n*T)*D)/Nt
            coeff.append([an,bn])
        return coeff                       

    def fourier_series(self,coeff,T):
        """fourier_series
        Make simple Fourier series with specified coefficients, 
        over time T. 
        T assumed to be in range [0,1).
        """
        i=1
        nmax=len(coeff)
        f=np.zeros(T.shape)
        #need a +1 somewhere due to 0-indexing, and not including constant
        for n in range(nmax):
            an, bn = coeff[n]
            f += an*np.cos(2*pi*(n+1)*T)+bn*np.sin(2*pi*(n+1)*T)
        return f

In [88]:
%pdb

Automatic pdb calling has been turned OFF


In [57]:
dem_sub  = dem[0:24*7*4]
temp_sub = 10*temp[0:24*7*4]

nmax = [0,4,4]

Df, Dcoeff = fourier_model.total_fourier_series(dem_sub,nmax)
plot_pred(Df,dem_sub,temp_sub)

AttributeError: 'builtin_function_or_method' object has no attribute 'dayofyear'

In [145]:
dem_scale.std()

1.0

In [294]:
%pdb

Automatic pdb calling has been turned ON


In [149]:
dem_scale,temp_scale,scale_df=scale_data(dem_sub,temp_sub)
plt.figure()
Dpred=Tmodel.param_fit(dem_scale,temp_scale)

Failed to hit tolerance after <built-in function iter> iter

Cost: 0.06780300101098831 0.06784870959652975


Cost, Old Cost = 0.06780300101098831,0.06784870959652975
Param: a0   -0.355393
ap    0.739753
an   -0.714479
Tp   -0.859248
Tn   -0.143258
dtype: float64
Param_grad: a0   -0.002524
ap    0.002695
an    0.010862
Tp    0.002000
Tn    0.007561
dtype: float64
Mean param Change 0.0006741380891686077 at iter 200


Cost, Old Cost = 0.07670151030736305,0.07687407174663803
Param: a0   -0.391960
ap    0.838400
an   -0.585513
Tp   -0.690012
Tn   -0.059845
dtype: float64
Param_grad: a0   -0.006662
ap    0.017932
an    0.016383
Tp    0.028599
Tn    0.008011
dtype: float64
Mean param Change 0.00224977889722755 at iter 100


Param: a0    1.831868e-16
ap    4.542121e-01
an    4.542121e-01
Tp    2.164935e-16
Tn    2.164935e-16
dtype: float64 



Init cost 1.0263110440225278


In [219]:
l,b,s=fit_init_params(dem_sub)

pred=predict_stl(l,b,s,dem_sub.index)

20 8


In [220]:
plt.plot(dem_sub.values,'b',temp_sub.values,'r',pred,'k')
plt.show()

1

Rambling Time!

From a Kalman filter perspective, I think that sometimes the error/innovation terms can be written as $\epsilon_t = y_t-\hat{y}_t$, where $y_t$ is the actual value, and $\hat{y}_t$ is the output of the model with no noise.  The innovation process, then gives a rule for updating (the set of parameters $\alpha,\beta,\Gamma, l, b, s_{i,t}$) how to change in the 

In [35]:
%pdb

Automatic pdb calling has been turned OFF
